# Data Split

We have decided to focus on the following question in our subsequent analysis:
- How does the proportion of high income individuals vary across sex?

In this document, we describe how the data is split. In particular, we stratify by sex. We ensure that the test data contains complete cases only to allow later comparison of different methods to deal with missingness. This is done in the following way:
- We perform a train/test split on the data with no missing values. The split is stratified by sex. We keep the test set thus obtained. This set is ensured to have complete data.
- We then take the remaining data (including the data which have missing values) to obtain the training set. 

This is shown below.

## Requirements

We begin by checking requirements.

In [1]:
######## REQUIREMENTS #################
### PLEASE RUN THIS CELL FIRST ########
#######################################

import json
import re

def extract_imports_from_code(code):
    # Regex patterns to extract import statements
    import_pattern = re.compile(r'^\s*import\s+([\w\.]+)', re.MULTILINE)
    from_import_pattern = re.compile(r'^\s*from\s+([\w\.]+)\s+import', re.MULTILINE)

    # Find all matches for both patterns
    imports = import_pattern.findall(code)
    from_imports = from_import_pattern.findall(code)

    # Return a set of unique imports
    return set(imports + from_imports)

def extract_imports_from_notebook(notebook_path):
    # Open and read the Jupyter notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    all_imports = set()

    # Iterate through each cell in the notebook
    for cell in notebook['cells']:
        if cell['cell_type'] == 'code':
            code = ''.join(cell['source'])  # Combine all lines of the code
            imports_in_cell = extract_imports_from_code(code)
            all_imports.update(imports_in_cell)

    return all_imports

notebook_path = '02.2-DataSplit.ipynb'  # Replace with your notebook file path
packages_to_install = extract_imports_from_notebook(notebook_path)
print("Packages to install:")
for package in sorted(packages_to_install):
    print(package)
print("\nSome packages like subprocess, os, re, sys, json are inbuilt so they don't need to be imported. We check and import the rest")


import subprocess
import sys

# List of required packages (excluding built-in modules)
required_packages = ['imblearn', 'numpy', 'pandas', 'ucimlrepo', 'scikit-learn']

# Function to check and install packages
def install(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

# Check and install required packages
for package in required_packages:
    try:
        __import__(package)
        print(f"'{package}' is already installed.")
    except ImportError:
        print(f"'{package}' not found. Installing...")
        install(package)

print("\nAll required packages are checked/installed.")

Packages to install:
imblearn.over_sampling
json
numpy
os
pandas
re
sklearn.model_selection
sklearn.preprocessing
subprocess
sys
ucimlrepo

Some packages like subprocess, os, re, sys, json are inbuilt so they don't need to be imported. We check and import the rest


'imblearn' is already installed.
'numpy' is already installed.
'pandas' is already installed.
'ucimlrepo' is already installed.
'scikit-learn' not found. Installing...

All required packages are checked/installed.


In [2]:
# Library imports
from sklearn.preprocessing import LabelEncoder  # For encoding categorical labels.
from imblearn.over_sampling import SMOTE       # For synthetic data generation.
import pandas as pd                             # For data manipulation and analysis.
from ucimlrepo import fetch_ucirepo            # To fetch datasets from UCI repository.
from sklearn.model_selection import train_test_split  # For splitting datasets into train/test sets.
import numpy as np                             # For numerical operations.
import os                                       # For operating system interactions.

## Data Split

We now do the required data split.

In [3]:
## SPLTTING PROCESS #######
## STEP 1: LOAD THE DATA ##
###########################

# Fetch the Adult dataset from UCI ML repository
adult = fetch_ucirepo(id=2)

# Note this takes up to 20 seconds to run

In [4]:
## SPLTTING PROCESS ###########
## STEP 2: PERFORM THE SPLIT ##
###############################

# Separate features and targets
X = adult.data.features
y = adult.data.targets

# Replace '?' with NaN in X
X.replace('?', np.nan, inplace=True)

# Clean the target variable (income)
y.loc[:, y.columns[0]] = y[y.columns[0]].replace({'<=50K.': '<=50K', '>50K.': '>50K'})

# Ensure the split maintains the proportion of 'sex' in the test set
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X, y, test_size=0.2, stratify=X['sex'], random_state=42
)

# Drop rows with missing data (NaN values) from X_test and ensure y_test is consistent
X_test = X_test_raw.dropna()
y_test = y_test_raw.loc[X_test.index]

# Subtract X_test from the original dataset to get X_train
X_train = X.drop(X_test.index)
y_train = y.drop(y_test.index)

# Display the proportion of 'sex' in the overall dataset, training, and test sets
overall_sex_proportion = X['sex'].value_counts(normalize=True)
train_sex_proportion = X_train['sex'].value_counts(normalize=True)
test_sex_proportion = X_test['sex'].value_counts(normalize=True)

# Print the proportions
print("Proportion of data by sex: overall data (before split), training data, test data.")
print("Overall sex proportion:\n", overall_sex_proportion)
print("Training set sex proportion:\n", train_sex_proportion)
print("Test set sex proportion:\n", test_sex_proportion)

C:\Users\megar\AppData\Local\Temp\ipykernel_31784\402134908.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace('?', np.nan, inplace=True)


Proportion of data by sex: overall data (before split), training data, test data.
Overall sex proportion:
 sex
Male      0.668482
Female    0.331518
Name: proportion, dtype: float64
Training set sex proportion:
 sex
Male      0.666466
Female    0.333534
Name: proportion, dtype: float64
Test set sex proportion:
 sex
Male      0.677327
Female    0.322673
Name: proportion, dtype: float64


We see that the data has been stratified by sex correctly, since we see that the training set and test set have approximately the same sex proportion as the overall data. We next look at the missing data.

In [5]:
# Count the number of missing values in X_train and X_test
missing_in_train = X_train.isna().sum()
missing_in_test = X_test.isna().sum()

# Count the total number of missing values in each dataset
total_missing_train = X_train.isna().sum().sum()
total_missing_test = X_test.isna().sum().sum()

# Print the number of missing data in each set
print("\nNumber of missing values in each feature (training set):")
print(missing_in_train)

print("\nNumber of missing values in each feature (test set):")
print(missing_in_test)


Number of missing values in each feature (training set):
age                  0
workclass         2799
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
dtype: int64

Number of missing values in each feature (test set):
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64


This looks reasonable. We now write the data to csv files below.

In [6]:
# Create a 'data' directory in the parent directory if it doesn't exist
os.makedirs("../data", exist_ok=True)

# Save X_train, y_train, X_test, and y_test to separate CSV files
X_train.to_csv("../data/X_train.csv", index=True)        # Save features of train set
y_train.to_csv("../data/y_train.csv", index=True)        # Save targets of train set
X_test.to_csv("../data/X_test.csv", index=True)          # Save features of test set
y_test.to_csv("../data/y_test.csv", index=True)          # Save targets of test set

"Datasets saved as '../data/X_train.csv', '../data/y_train.csv', '../data/X_test.csv', and '../data/y_test.csv' with index."

"Datasets saved as '../data/X_train.csv', '../data/y_train.csv', '../data/X_test.csv', and '../data/y_test.csv' with index."

This code can be used to load the data in subsequent analyses (and optionally check with the output to make sure it worked).

In [7]:
# Load the saved datasets from the CSV files
X_train = pd.read_csv("../data/X_train.csv", index_col=0)  # Use the first column as index
y_train = pd.read_csv("../data/y_train.csv", index_col=0)  # Use the first column as index
X_test = pd.read_csv("../data/X_test.csv", index_col=0)    # Use the first column as index
y_test = pd.read_csv("../data/y_test.csv", index_col=0)    # Use the first column as index

# Display the first few rows of the loaded datasets
(X_train.head(), y_train.head(), X_test.head(), y_test.head())


(   age         workclass  fnlwgt  education  education-num  \
 0   39         State-gov   77516  Bachelors             13   
 1   50  Self-emp-not-inc   83311  Bachelors             13   
 2   38           Private  215646    HS-grad              9   
 3   53           Private  234721       11th              7   
 4   28           Private  338409  Bachelors             13   
 
        marital-status         occupation   relationship   race     sex  \
 0       Never-married       Adm-clerical  Not-in-family  White    Male   
 1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
 2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
 3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
 4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   
 
    capital-gain  capital-loss  hours-per-week native-country  
 0          2174             0              40  United-States  
 1             0             0          

# Appendix: Checking

Some checks were done to ensure that everything was correct. This appendix may be skipped. 

In [8]:
# Load the saved datasets from the CSV files
X_train_loaded = pd.read_csv("../data/X_train.csv", index_col=0)  # Use the first column as index
y_train_loaded = pd.read_csv("../data/y_train.csv", index_col=0)  # Use the first column as index
X_test_loaded = pd.read_csv("../data/X_test.csv", index_col=0)    # Use the first column as index
y_test_loaded = pd.read_csv("../data/y_test.csv", index_col=0)    # Use the first column as index

# Display the first few rows of the loaded datasets
(X_train_loaded.head(), y_train_loaded.head(), X_test_loaded.head(), y_test_loaded.head())

# Compare loaded data with original data
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X, y, test_size=0.2, stratify=X['sex'], random_state=42
)
X_test = X_test_raw.dropna()
y_test = y_test_raw.loc[X_test.index]
X_train = X.drop(X_test.index)
y_train = y.drop(y_test.index)

# Check if the original and loaded training feature sets are equal
X_train_comparison = X_train_loaded.equals(X_train)
y_train_comparison = y_train_loaded.equals(y_train)

# Check if the original and loaded test feature sets are equal
X_test_comparison = X_test_loaded.equals(X_test)
y_test_comparison = y_test_loaded.equals(y_test)

# Create a summary of the comparisons
comparison_results = {
    'X_train_equal': X_train_comparison,
    'y_train_equal': y_train_comparison,
    'X_test_equal': X_test_comparison,
    'y_test_equal': y_test_comparison,
}

# Print out the comparison results
for key, value in comparison_results.items():
    print(f"{key}: {value}")

# If any discrepancies found, print the first few mismatched rows
if not X_train_comparison:
    print("\nMismatched rows in X_train:")
    print(X_train_loaded[X_train_loaded != X_train].dropna())

if not y_train_comparison:
    print("\nMismatched values in y_train:")
    print(y_train_loaded[y_train_loaded != y_train].dropna())

if not X_test_comparison:
    print("\nMismatched rows in X_test:")
    print(X_test_loaded[X_test_loaded != X_test].dropna())

if not y_test_comparison:
    print("\nMismatched values in y_test:")
    print(y_test_loaded[y_test_loaded != y_test].dropna())


X_train_equal: True
y_train_equal: True
X_test_equal: True
y_test_equal: True


# Appendix: Extension (SMOTE)

Since we have unbalanced data, we may want to use a technique to balance the classes. One such method is SMOTE (synthetic minority oversampling technique) which uses nearest-neighbour methods to create synthetic minority samples (see Section 5.2 for details).

In [9]:
# Initialize label encoders for each categorical column
label_encoders = {}
categorical_columns = X_train.select_dtypes(include=['object']).columns  # Get categorical columns

# Apply label encoding to categorical columns in X_train
X_train_encoded = X_train.copy()  # Make a copy of the original data
for col in categorical_columns:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train_encoded[col])
    label_encoders[col] = le  # Save the encoder for each column

# Apply SMOTE to the label-encoded training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_encoded, y_train)

# After SMOTE, reverse the label encoding to restore original categorical values
X_train_smote_restored = X_train_smote.copy()
for col in categorical_columns:
    le = label_encoders[col]  # Get the original label encoder for the column
    X_train_smote_restored[col] = le.inverse_transform(X_train_smote_restored[col])

# Save the SMOTE-applied datasets (with restored categorical values) to CSV files
X_train_smote_restored.to_csv("../data/X_train_smote.csv", index=True)  # Save features of SMOTE train set
y_train_smote.to_csv("../data/y_train_smote.csv", index=True)           # Save targets of SMOTE train set

print("SMOTE-applied datasets (with restored categories) saved as '../data/X_train_smote.csv' and '../data/y_train_smote.csv'.")

SMOTE-applied datasets (with restored categories) saved as '../data/X_train_smote.csv' and '../data/y_train_smote.csv'.


In [10]:
# Load the SMOTE-applied datasets
X_train_smote_loaded = pd.read_csv("../data/X_train_smote.csv", index_col=0)  # Use the first column as index
y_train_smote_loaded = pd.read_csv("../data/y_train_smote.csv", index_col=0)  # Use the first column as index

# Display the first few rows of the datasets
print("X_train_smote Loaded from CSV:")
print(X_train_smote_loaded.head())

print("\ny_train_smote Loaded from CSV:")
print(y_train_smote_loaded.head())

X_train_smote Loaded from CSV:
   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country  
0          2174             0              40  United-States  
1             0          

In [11]:
# Check the distribution of the target variable y_train before and after SMOTE
y_train_distribution_before = y_train.value_counts(normalize=True)
y_train_smote_distribution_after = y_train_smote.value_counts(normalize=True)

y_train_distribution_before, y_train_smote_distribution_after

(income
 <=50K     0.761628
 >50K      0.238372
 Name: proportion, dtype: float64,
 income
 <=50K     0.5
 >50K      0.5
 Name: proportion, dtype: float64)